In [1]:
"""
调参技巧
"""
# 可视化
# fine-tune
# 优化算法、激活函数、初始化函数
# 批归一化
# 数据增强

In [2]:
# tensorboard
# 1. 指定面板图上显示的变量
# 2. 训练过程中将这些变量计算出来，输出到文件中
# 3. 文件解析 ./tensorboard --logdir=dir.

In [3]:
import tensorflow as tf
import numpy as np
import pickle
import os

CIFAR_DIR = './../cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [4]:
def load_data(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1  # 为了提高预测准确率，需要对数据进行归一化
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0  # 当前数据集指明遍历的位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)  # 索引进行混排
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """返回batch_size个样本"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [5]:
# 计算图模型，先新建计算图，然后将数据塞入计算图中
# placeholder 占位符的作用
x = tf.placeholder(tf.float32, [None, 3072])  # None 样本数不确定，mini_batch
# [None]
y = tf.placeholder(tf.int64, [None])

# 图片转换，reshape
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 大小： 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# 第一个卷积层
# 参数说明
# - 32: 卷积层输出的通道数目
# - (3, 3): 卷积核的大小
# - padding: 类型有两种 same|valid，same的话使得输出的图像大小不会变化
# - activation: 激活函数
# - name: 给这一层做一个命名，将图打印出来有意义的的一个图
# conv1: 神经元图，feature_map, 输出图像
conv1_1 = tf.layers.conv2d(x_image, 32, (3, 3), padding='same', activation=tf.nn.relu, name='conv1_1')

conv1_2 = tf.layers.conv2d(conv1_1, 32, (3, 3), padding='same', activation=tf.nn.relu, name='conv1_2')

# pooling 池化层
# 参数说明
# - conv1: 输入参数
# - (2, 2): kernal size
# - (2, 3): 步长 (stride)
# - name: 给这一层做一个命名，将图打印出来有意义的的一个图
# 大小： 16*16
pooling1 = tf.layers.max_pooling2d(conv1_2, (2, 2), (2, 2), name='pool1')

conv2_1 = tf.layers.conv2d(pooling1, 32, (3, 3), padding='same', activation=tf.nn.relu, name='conv2_1')

conv2_2 = tf.layers.conv2d(conv2_1, 32, (3, 3), padding='same', activation=tf.nn.relu, name='conv2_2')
# 大小： 8*8
pooling2 = tf.layers.max_pooling2d(conv2_2, (2, 2), (2, 2), name='pool2')

conv3_1 = tf.layers.conv2d(pooling2, 32, (3, 3), padding='same', activation=tf.nn.relu, name='conv3_1')
conv3_2 = tf.layers.conv2d(conv3_1, 32, (3, 3), padding='same', activation=tf.nn.relu, name='conv3_2')
# 大小： 4*4*32
pooling3 = tf.layers.max_pooling2d(conv3_2, (2, 2), (2, 2), name='pool3')

# 做一个展平
# 展平后的shape: [None, 4*4*32]
flatten = tf.layers.flatten(pooling3)

# 全连接层，使它能够映射到10个类别上去
y_ = tf.layers.dense(flatten, 10)

# 交叉熵损失数函数
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# 执行原理
# y_ -> softmax
# y -> one_hot
# loss -> ylogy_

# 寻找最大概率值的 indices
# 1 代表在第一个维度上
predict = tf.argmax(y_, 1)
# correct_prediction 结果可能是[0,1,0,0,1,1,1]
correct_prediction = tf.equal(predict, y)
# 求准确率，就是求平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

# 在我们有了目标函数和损失函数，再定义梯度下降方法
# 梯度下降的变种
# 优化最小loss的值
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
# 计算图构建完成

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.MaxPooling2D instead.


Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
def variable_summary(var, name):
    """
    给一个变量的很多统计量建立summary
    var: 给它计算summary的变量
    name: 命名空间
    """
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.histogram('histogram', var)

with tf.name_scope('summary'):
    variable_summary(conv1_1, 'conv1_1')
    variable_summary(conv1_2, 'conv1_2')
    variable_summary(conv2_1, 'conv2_1')
    variable_summary(conv2_2, 'conv2_2')
    variable_summary(conv3_1, 'conv3_1')
    variable_summary(conv3_2, 'conv3_2')

loss_summary = tf.summary.scalar('loss', loss)
# 'loss': <10, 1.1>, <20, 1.08>
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

source_image = (x_image + 1) * 127.5
inputs_summary = tf.summary.image('inputs_image', source_image)

merged_summary = tf.summary.merge_all()
merged_summary_test = tf.summary.merge([loss_summary, accuracy_summary])

LOG_DIR = '.'
run_label = './run_vgg_tensorboard'
run_dir = os.path.join(LOG_DIR, run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)
train_log_dir = os.path.join(run_dir, 'train')
test_log_dir = os.path.join(run_dir, 'test')
if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)

In [7]:
# 定义 init函数，执行初始化
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

output_summary_every_steps = 100

# session，打开session后执行计算图
with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter(train_log_dir, sess.graph)
    test_writer = tf.summary.FileWriter(test_log_dir)
    fixed_test_batch_data, fixed_test_batch_labels = test_data.next_batch(batch_size)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        eval_ops = [loss, accuracy, train_op]
        should_output_summary = ((i+1) % output_summary_every_steps == 0)
        if should_output_summary:
            eval_ops.append(merged_summary)
        eval_ops_results = sess.run(eval_ops, feed_dict={x: batch_data, y: batch_labels})
        loss_val, acc_val = eval_ops_results[0:2]
        if should_output_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str, i+1)
            test_summary_str = sess.run([merged_summary_test], feed_dict={x: fixed_test_batch_data, y: fixed_test_batch_labels})[0]
            test_writer.add_summary(test_summary_str, i+1)
        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' % (i+1, loss_val, acc_val))
        if (i+1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict={x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step: 100, loss: 1.88112, acc: 0.30000
[Train] Step: 200, loss: 1.69259, acc: 0.40000
[Train] Step: 300, loss: 1.43113, acc: 0.45000
[Train] Step: 400, loss: 1.65907, acc: 0.55000
[Train] Step: 500, loss: 1.58422, acc: 0.40000
[Train] Step: 600, loss: 1.51464, acc: 0.55000
[Train] Step: 700, loss: 1.26765, acc: 0.60000
[Train] Step: 800, loss: 1.05161, acc: 0.75000
[Train] Step: 900, loss: 1.17924, acc: 0.65000
[Train] Step: 1000, loss: 1.71907, acc: 0.50000
(10000, 3072)
(10000,)
[Test ] Step: 1000, acc: 0.52250
[Train] Step: 1100, loss: 1.04993, acc: 0.65000
[Train] Step: 1200, loss: 1.44526, acc: 0.40000
[Train] Step: 1300, loss: 1.16413, acc: 0.55000
[Train] Step: 1400, loss: 1.28927, acc: 0.60000
[Train] Step: 1500, loss: 1.24894, acc: 0.70000
[Train] Step: 1600, loss: 1.22816, acc: 0.55000
[Train] Step: 1700, loss: 1.21649, acc: 0.60000
[Train] Step: 1800, loss: 1.35005, acc: 0.60000
[Train] Step: 1900, loss: 1.29912, acc: 0.35000
[Train] Step: 2000, loss: 1.17310, acc: 0